In [2]:
#User Items
#Importo Librerias
import pandas as pd
import ast
import gzip

In [3]:
# User_Items
List_User_Items = []
#with gzip.open(u"/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/users_items.json.gz", 'rt', encoding='UTF-8') as f:
with gzip.open(u"./Dataset/users_items.json.gz", 'rt', encoding='UTF-8') as f:
#with open("/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/user_reviews.json.gz", encoding='UTF-8') as f:
    for line in f.readlines():
        List_User_Items.append(ast.literal_eval(line))

df_ui = pd.DataFrame(List_User_Items)
df_ui = df_ui.explode('items').reset_index()
df_ui = df_ui.drop(columns="index")
df_ui = pd.concat([df_ui, pd.json_normalize(df_ui['items'])], axis=1)
df_ui = df_ui.drop(columns='items')

In [5]:
##Busqueda de item_id nulos
df_ui[df_ui['item_id'].isna()]

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
3733,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN,NaN
3849,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
6019,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN,NaN
6523,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN,NaN
7237,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5169470,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169471,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169472,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN,NaN


In [6]:
# Elimino Vacios
df_ui.dropna(inplace=True)

In [21]:
#Valido los nulos
df_ui.isnull().sum()

user_id             0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

In [8]:
# Elimino columnas no necesarias
df_ui.drop(columns=['items_count','steam_id','user_url','playtime_2weeks'],inplace=True)

In [9]:
# Seteo el indice
df_ui.reset_index(drop=True, inplace=True)

In [10]:
# Convierto id y playtime_forever a entero
df_ui['playtime_forever'] = df_ui['playtime_forever'].astype('int')
df_ui['item_id'] = df_ui['item_id'].astype('int')

In [11]:
# Muestro Dataframe
df_ui.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [12]:
#Elimino los play_time forever(año) 0
df_ui.drop(df_ui[df_ui['playtime_forever'] == 0].index, inplace=True)
df_ui.reset_index(drop=True, inplace=True)

In [20]:
# Muestro Dataframe
df_ui.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,30,Day of Defeat,7
2,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,3830,Psychonauts,333


In [17]:
# Guardo Dataframe en csv comprimido
# df_ui.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ui.csv.gz',index=False, compression='gzip')
df_ui.to_csv('./Dataset/df_ui.csv.gz',index=False, compression='gzip')

In [53]:
#Crear archivo base de genres
df_sgui=pd.merge(df_sg[['item_id','genres','release_date']],df_ui[['item_id','playtime_forever']],on='item_id',how='left')

#df_ug=pd.merge(df_sg[['genres','release_date']],df_ui[['playtime_forever']],on='item_id',how='left')
#df_sgui
#df_ug[df_ug['item_id']==10]

# hago un merge teniendo todos los usuarios
#user_rev_item = pd.merge(user_items_proc[['user_id','item_id','playtime_forever']],user_review[['user_id','recommend']],on = 'user_id',how='left')
# Me quedo solo con los que estan nulos en reviews
#user_rev_item = user_rev_item[user_rev_item['recommend'].isnull()]
# Hago un agrupamiento por usuario y saco el id del que tiene mas minuntos.
#max_playtime_items = user_rev_item.groupby('user_id')['playtime_forever'].idxmax()
#result_df = user_rev_item.loc[max_playtime_items, ['user_id', 'item_id']]
#result_df.to_csv('../datasets/user_item_max_horas.csv',index=False) # Exporto para ser usado por la funcion

In [ ]:
 # hago un merge teniendo todos los usuarios
user_rev_item = pd.merge(df_ui[['user_id','item_id','playtime_forever']],user_review[['user_id','recommend']],on = 'user_id',how='left')
# Me quedo solo con los que estan nulos en reviews
user_rev_item = user_rev_item[user_rev_item['recommend'].isnull()]
# Hago un agrupamiento por usuario y saco el id del que tiene mas minuntos.
max_playtime_items = user_rev_item.groupby('user_id')['playtime_forever'].idxmax()
result_df = user_rev_item.loc[max_playtime_items, ['user_id', 'item_id']]
result_df.to_csv('../datasets/user_item_max_horas.csv',index=False) # Exporto para ser usado por la funcion